<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigdataII_11_CustomDataset_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/Lecture_BigData"

# 第11回 (Part2) カスタムデータによる画像認識

## 0. 事前準備

In [ ]:
# 必要ライブラリ・コマンドの導入

!pip install japanize_matplotlib | tail -n 1
!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1
w = !apt install tree
print(w[-2])

In [ ]:
# 必要ライブラリのインポート

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
# PyTorch関連ライブラリのインポート

import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from torchviz import make_dot
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets

In [ ]:
# warning表示off
import warnings
warnings.simplefilter('ignore')

# デフォルトフォントサイズ変更
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
plt.rcParams['axes.grid'] = True

# numpyの表示桁数設定
np.set_printoptions(suppress=True, precision=5)

In [ ]:
# GPUチェック
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# 共通関数のダウンロード
!git clone https://github.com/makaishi2/pythonlibs.git

# 共通関数のロード
from pythonlibs.torch_lib1 import *

# 共通関数の存在チェック
print(README)

## 1 transformsを用いたデータ拡張

参照：https://pystyle.info/pytorch-list-of-transforms/

データ拡張（Data Augmentation）は、元のデータセットから新しいトレーニングデータを生成する方法で、主に以下の目的で用いられる.

1. **多様性を増やす**：限られたデータセットから、多様なパターンや変化を持つデータを生成することで、モデルの汎化性能を向上させる．
   
2. **過学習を防ぐ**：モデルがトレーニングデータに過度に適応することを防ぎ、新しいデータに対する汎化性能を高める．

画像、音声、テキストなどのデータ形式に対して様々な手法がある．
例えば画像の場合

- **反転（フリップ）**: 画像を水平または垂直に反転させる。
- **回転**: 画像を一定の角度で回転させる。
- **ズーム**: 画像の一部を拡大または縮小する。
- **クロッピング**: 画像の一部を切り取る。
- **色の変更**: 画像の色合いや明るさを変える。
- **ノイズの追加**: 画像にノイズを追加する。

これらの手法を使用して、元のデータを変換し、新しいデータを生成することで、データセットのサイズを増やしたり、さまざまなバリエーションを持つデータを作成する．これにより、機械学習モデルがより多くのパターンを学習し、一般化能力を向上させることが期待される．



グレースケール

In [ ]:
from PIL import Image
from torch.utils import data as data
from torchvision import transforms as transforms

img = Image.open("sample.jpg")
display(img)

# グレースケール変換を行う Transforms
transform = transforms.Grayscale()

# 関数呼び出しで変換を行う
img = transform(img)
img

中心を切り抜く

* size – 切り抜く大きさ
 * int – 幅 size、高さ size となるように切り抜く
 * 2-ints sequence – 幅 size[0]、高さ size[1] となるように切り抜


In [ ]:
img = Image.open("sample.jpg")
transform = transforms.CenterCrop(150)

img = transform(img)
img

同じ大きさの5枚の画像をタイル上に並べる。

In [ ]:
def tile_imgs(imgs, n_cols=3):
    """同じ大きさの複数枚の画像をタイル上に並べる。
    """
    n_rows = int(np.ceil(len(imgs) / n_cols))
    w, h = imgs[0].size

    # 結合後の画像
    concat_img = Image.new("RGB", (w * n_cols, h * n_rows))

    for i, img in enumerate(imgs):
        row, col = i % n_cols, i // n_cols
        concat_img.paste(img, (w * row, h * col))

    return concat_img

img = Image.open("sample.jpg")
transform = transforms.FiveCrop(150)

imgs = transform(img)
tile_imgs(imgs)

ランダムクロップ（ランダムに切り抜く）

In [ ]:
import random

img = Image.open("sample.jpg")
random.seed(0)

transform = transforms.RandomCrop(150)
display(transform(img))

transform = transforms.RandomCrop((100, 200))
display(transform(img))

transform = transforms.RandomCrop(250, pad_if_needed=True)
display(transform(img))

リサイズ

In [ ]:
img = Image.open("sample.jpg")

transform = transforms.Resize(150)
display(transform(img))

transform = transforms.Resize((150, 150))
display(transform(img))

ガウシアンフィルタを行う Transform です。

In [ ]:
img = Image.open("sample.jpg")

transform = transforms.GaussianBlur(kernel_size=5)
display(transform(img))

反転

In [ ]:
img = Image.open("sample.jpg")
random.seed(0)

transform = transforms.RandomHorizontalFlip(0.9)
display(transform(img))

In [ ]:
img = Image.open("sample.jpg")
random.seed(0)

transform = transforms.RandomVerticalFlip(0.9)
display(transform(img))

回転

In [ ]:
img = Image.open("sample.jpg")
random.seed(0)

transform = transforms.RandomRotation(degrees=15)

imgs = [transform(img) for _ in range(6)]
tile_imgs(imgs)

**Compose**

Compose を使用すると、複数の Transform を連続して行う Transform を作成できます。画像を読み込む際にリサイズや標準化など一連の処理を行いたい場合に便利です。

(256, 256) にリサイズする
画像の中心を (224, 224) で切り抜く
PIL Image をテンソルに変換する

In [ ]:
img = Image.open("sample.jpg")

transform = transforms.Compose(
    [transforms.Resize(256),
     transforms.CenterCrop(224),
     transforms.ToTensor()]
)

# 関数呼び出しで変換を行う
img = transform(img)
print(type(img), img.shape)

## 2 DataLoaderの設定

### データダウンロード・解凍

シベリアンハスキーとオオカミの画像を利用  
ダウンロード元  
https://pixabay.com/ja/

In [ ]:
# データダウンロード
w = !wget https://github.com/makaishi2/pythonlibs/raw/main/images/dog_wolf.zip
print(w[-2])

# 解凍
!unzip dog_wolf.zip | tail -n 1

# 解凍結果のツリー表示
!tree dog_wolf

### データ拡張・Transforms定義

In [ ]:
# 検証データ用 : 正規化のみ実施
test_transform = transforms.Compose([
    ***here *** # リサイズ
    ***here *** # クロップ
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

# 訓練データ用: 正規化に追加で反転とRandomErasingを実施 ← より多様な画像を用いて学習させるねらい
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    ***here *** # リサイズ
    ***here *** # クロップ
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
])

### データセット定義

In [ ]:
# データセット定義
data_dir = 'dog_wolf'

import os
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

classes = ['dog', 'wolf']

train_data = datasets.ImageFolder( ***here *** ) # 学習用
test_data = datasets.ImageFolder( ***here *** )  # 検証用

In [ ]:
# データ件数確認

print(f'学習データ: {len(train_data)}件')
print(f'検証データ: {len(test_data)}件')

### DataLoader定義

In [ ]:
batch_size = 5
# 学習データ
train_loader = DataLoader( ***here *** )

# 検証データ
test_loader = DataLoader( ***here *** )

# イメージ表示用
train_data2 = datasets.ImageFolder(train_dir, transform=test_transform)
train_loader2 = DataLoader(train_data2, batch_size=40, shuffle=False)
test_loader2 = DataLoader(test_data, batch_size=10, shuffle=True)

### イメージ表示

In [ ]:
# 訓練用データ(４0件)
show_images_labels(train_loader2, classes, None, None)

In [ ]:
# 検証用データ(10件)
torch_seed()
show_images_labels(test_loader2, classes, None, None)

## 3 モデル

### 3.1 MLP (Multi Layer Perceptron)

In [ ]:
class Net (nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(224 * 224  * 3, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, len(classes))
        self.dropout1 = nn.Dropout2d(0.2)
        self.dropout2 = nn.Dropout2d(0.2)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        return self.relu(self.fc3(x))

net = Net()
net.cuda()  # GPU対応
print(net)

学習の定義

In [ ]:
# 乱数初期化
torch_seed()

# AdaptiveAvgPool2d関数の取り外し
net.avgpool = nn.Identity()

# GPUの利用
net = net.to(device)

lr = 0.001
# 損失関数定義
criterion = nn.CrossEntropyLoss()

# 最適化関数定義
# パラメータ修正の対象を最終ノードに限定
optimizer = optim.SGD(net.parameters(),lr=lr,momentum=0.9)

# historyファイルも同時に初期化する
history = np.zeros((0, 5))

学習の実行

※ fit関数を利用する（学習の実装は省略）

In [ ]:
num_epochs = 10
history = fit(net, optimizer, criterion, num_epochs, train_loader, test_loader, device, history)

結果表示

In [ ]:
evaluate_history(history)

テストデータの分類結果表示

In [ ]:
torch_seed()
show_images_labels(test_loader2, classes, net, device)

### 3.2 CNN (Convolutional Neural Network）

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)  # 畳み込み層:(入力チャンネル数, フィルタ数、フィルタサイズ)
        self.relu = nn.ReLU()  # ReLU
        self.pool = nn.MaxPool2d(2, 2)  # プーリング層:（領域のサイズ, 領域の間隔）
        self.conv2 = nn.Conv2d(8, 16, 5)
        self.fc1 = nn.Linear(16*53*53, 256)  # 全結合層
        self.dropout = nn.Dropout(p=0.5)  # ドロップアウト:(p=ドロップアウト率)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, self.num_flat_features(x))
        #x = x.view(-1, 16*5*5)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
net.cuda()  # GPU対応
print(net)

学習の定義

In [ ]:
# 乱数初期化
torch_seed()

# AdaptiveAvgPool2d関数の取り外し
net.avgpool = nn.Identity()

# GPUの利用
net = net.to(device)

lr = 0.001
# 損失関数定義
criterion = nn.CrossEntropyLoss()

# 最適化関数定義
# パラメータ修正の対象を最終ノードに限定
optimizer = optim.SGD(net.parameters(),lr=lr,momentum=0.9)

# historyファイルも同時に初期化する
history = np.zeros((0, 5))

学習の実行

In [ ]:
num_epochs = 10
history = fit(net, optimizer, criterion, num_epochs,
          train_loader, test_loader, device, history)

結果表示

In [ ]:
evaluate_history(history)

テストデータの分類結果表示

In [ ]:
torch_seed()
show_images_labels(test_loader2, classes, net, device)

### 3.3 学習済みモデルを活用 (転移学習)

使用可能なモデルのリンク

In [ ]:
# 学習済みモデルの読み込み
from torchvision import models

net = models.vgg19_bn(pretrained = True) # VGG19

for param in net.parameters():
    param.requires_grad = False

# 乱数初期化
torch_seed()

# 最終ノードの出力を2に変更する
in_features = net.classifier[6].in_features
net.classifier[6] = nn.Linear(in_features, 2)

# AdaptiveAvgPool2d関数の取り外し
net.avgpool = nn.Identity()

# GPUの利用
net = net.to(device)

lr = 0.001
# 損失関数定義
criterion = nn.CrossEntropyLoss()

# 最適化関数定義
# パラメータ修正の対象を最終ノードに限定
optimizer = optim.SGD(net.classifier[6].parameters(),lr=lr,momentum=0.9)

# historyファイルも同時に初期化する
history = np.zeros((0, 5))

In [ ]:
# 学習の実行

# 結果表示

# テストデータの分類結果表示

## 演習１：その他のサンプルデータ

ハチとアリの画像データセット

In [ ]:
# サンプルデータのダウンロード
w = !wget -nc https://download.pytorch.org/tutorial/hymenoptera_data.zip

# 結果確認
print(w[-2])

In [ ]:
# データ解凍
w = !unzip -o hymenoptera_data.zip

# 結果確認
print(w[-1])

In [ ]:
# 解凍ファイルのtree表示
!tree hymenoptera_data

## 演習２：自前のデータセット

datasetのフォルダの中に、trainとtestフォルダを作成し、それぞれのフォルダに画像を用意する